In [2]:
import os
import ast
import json
import string
import openai
import numpy as np
import pandas as pd

from tqdm import tqdm
from langchain import PromptTemplate

from src.DST.evaluate_utils import remapping
from src.DST.dst import SLOTS_DESCRIPTIONS, SLOTS_REVERSE_REMAPPING
from src.config import CONFIG

from dataclasses import dataclass, field
from typing import Optional
from transformers import TrainingArguments
from src.DST.evaluate_utils import unpack_belief_states, fix_typos, nested_fix, remapping


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)



@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to utilize.
    """
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={"help": "The path of the HuggingFace model."}
    )
    model_name_or_path_agent: Optional[str] = field(
        default=None,
        metadata={"help": "The path of the HuggingFace model for the agent"}
    )
    use_int8: Optional[bool] = field(
        default=False,
        metadata={"help": "Whether to use int8 model or not."}
    )
    use_deepspeed: Optional[bool] = field(
        default=False,
        metadata={"help": "Whether to use deepspeed model or not."}
    )
    
    

@dataclass
class DataArguments:
    """
    Arguments pertaining to the data loading and preprocessing pipeline.
    """
    dataset_name: Optional[str] = field(
        default=None,
        metadata={"help": "Train dataset path"}
    )
    dataset_names: Optional[str] = field(
        default=None,
        metadata={"help": "Train dataset paths"}
    )
    root_data_path: Optional[str] = field(
        default="./data", metadata={"help": "The path to the data directory."},
    )
    mwoz_path: Optional[str] = field(
        default="/home/willy/instructod/MultiWOZ_2.1/",
        metadata={"help": "MWOZ path"}
    )
    dialog_history_limit_dst: Optional[int] = field(
        default=0,
        metadata={"help": "Lenght of dialogue history for dst"}
    )
    dialog_history_limit_dst_recorrect: Optional[int] = field(
        default=0,
        metadata={"help": "Lenght of dialogue history for dst update"}
    )
    dialog_history_limit_rg: Optional[int] = field(
        default=20,
        metadata={"help": "Lenght of dialogue history for response generation"}
    )
    dialog_history_limit_e2e: Optional[int] = field(
        default=20,
        metadata={"help": "Lenght of dialogue history for e2e"}
    )
    single_domain_only: Optional[bool] = field(
        default=False,
        metadata={"help": "Whether to keep only the single domain sample or not"}
    )
    with_slot_description: Optional[bool] = field(
        default=False,
        metadata={"help": "Whether to use slot description or not for DST"}
    )
    with_slot_domain_diff: Optional[bool] = field(
        default=False,
        metadata={"help": "differentiation between slot and domain"}
    )
    with_all_slots: Optional[bool] = field(
        default=True,
        metadata={"help": "Whether to use all slots or not"}
    )
    debug_mode: Optional[bool] = field(
        default=False,
        metadata={"help": "debug mode to only try 20 samples"}
    )
    start_idx: Optional[int] = field(
        default=0,
        metadata={"help": "Starting index to restart the prediction if needed"}
    )
    save_path: Optional[str] = field(
        default="results/",
        metadata={"help": "save path"}
    )
    save_every: Optional[int] = field(
        default=5,
        metadata={"help": "every step to save in case api fail"}
    )
    db_format_type: Optional[str] = field(
        default="1",
        metadata={"help": "1 is more precise, 2 is more concise for db integration"},
    )
    load_path: Optional[str] = field(
        default="results/",
        metadata={"help": "load path"}
    )
    agent_max_iterations: Optional[int] = field(
        default=5,
        metadata={"help": "Max number of iterations for agents in e2e (higher=better but more expensive)"}
    )
    verbose: Optional[bool] = field(
        default=False,
        metadata={"help": "verbosity for agent call in database retrieval"}
    )
    do_inference: Optional[bool] = field(
        default=False,
        metadata={"help": "use to do inference with the e2e agent setting"}
    )
    accumulate_bs: Optional[bool] = field(
        default=False,
        metadata={"help": "evaluation setting to accumulate all turn-level bs"}
    )   
    with_slot_filtering: Optional[bool] = field(
        default=False,
        metadata={"help": "slot filtering during DST eval (filter non-existent slots)"}
    )   
    

@dataclass
class PromptingArguments(TrainingArguments):
    """
    Arguments pertraining to the prompting pipeline.
    """
    output_dir: Optional[str] = field(
        default="./out",
        metadata={"help": "Output directory"},
    )
    task: Optional[str] = field(
        default="dst",
        metadata={"help": "Task to perform"}
    )
    max_requests_per_minute: Optional[int] = field(
        default=20,
        metadata={"help": "Max number of requests for OpenAI API."}
    )
    openai_api_key_name: Optional[str] = field(
        default="OPENAI_API_KEY",
        metadata={"help": "OpenAI API key name."}
    )

class PromptConstructor():
    def __init__(self, 
                 config):
        self.config = config
        self.instructions = config["INSTRUCTIONS"]
        self.prompt_templates = config["PROMPT_TEMPLATES"]
        self.examples = config["EXAMPLES"]
        
    def _get_slots_from_domains(self, domains, ontology, with_slot_description, with_all_slots, with_slot_domain_diff):
        
        if with_all_slots:
            domains = ["restaurant", "train", "attraction", "hotel", "taxi"]
        
        slots = []
        for slot in list(ontology.keys()):
            splitted_slot = slot.split("-")
            if splitted_slot[0] in domains:
                if with_slot_domain_diff:
                    if splitted_slot[-1] not in slots:
                        slots.append(splitted_slot[-1])
                else:
                    slots.append(splitted_slot[0] + "-" + splitted_slot[-1])
        
        slots_info = []
        added_slots = []
        if with_slot_description:
            for slot in slots:
                splitted_slot = slot.split("-")
                if with_slot_domain_diff:
                    if slot in added_slots:
                        continue
                    slots_info.append(f"name: {slot}, description: {SLOTS_DESCRIPTIONS[slot.lower()]}")
                    added_slots.append(slot)
                else:
                    slots_info.append(f"name: {slot}, description: {SLOTS_DESCRIPTIONS[splitted_slot[1].lower()]}")

                    
            slots = slots_info
        
        slots_prompt = "\n".join(slots)
        if with_slot_domain_diff:
            return slots_prompt + f"\n\nDOMAINS: {', '.join(domains)}"
        else:
            return slots_prompt
            
                
                
#         if with_all_slots:
#             domains = "all"
        
#         if with_slot_description:
#             with_req_inf_differentiation = False #Slot description is the discriminator

#         if domains == "all":
#             if with_req_inf_differentiation:
#                 req_slots = ", ".join(self.config["multiwoz21"]["all_requestable_slots"])
#                 inf_slots = ", ".join(self.config["multiwoz21"]["all_informable_slots"])
#             else:
#                 slots = set(self.config["multiwoz21"]["all_requestable_slots"] + 
#                             self.config["multiwoz21"]["all_informable_slots"])
#                 slots = ", ".join(slots)
#         elif not isinstance(domains, list):
#             raise ValueError("""Provided domain should be either 'all' or list of valid domain names:
#                                 - for multiwoz2.1 and 2.4: taxi, restaurant, hotel, train, attraction""")
#         else:
#             req_slots = ""
#             inf_slots = ""
#             domain_req_slots = []
#             domain_inf_slots = []
#             for domain in domains:
#                 domain_req_slots += self.config["multiwoz21"]["requestable_slots"][domain]
#                 domain_inf_slots += self.config["multiwoz21"]["informable_slots"][domain]
#             if with_req_inf_differentiation:
#                 domain_req_slots = set(domain_req_slots)
#                 domain_inf_slots = set(domain_inf_slots)
#                 req_slots += ", ".join(domain_req_slots)
#                 inf_slots += ", ".join(domain_inf_slots)
#             else:
#                 slots = set(domain_req_slots + domain_inf_slots)
#                 slots = ", ".join(slots)

#         if with_req_inf_differentiation:
#             slots_info = f"Requestable slots: {req_slots}\nInformable slots: {inf_slots}"
#         else:
#             slots_info = f"{slots}"

#         if with_slot_description:
#             slots = slots.split(", ")
#             slots_info = ""
#             for slot in slots:
#                 if slot not in self.config["multiwoz21"]["all_informable_slots"]:
#                     continue
#                 slots_info += f"name: {slot}, description: {SLOTS_DESCRIPTIONS[slot]}\n"
#             slots_info = slots_info[:-2]
        
#         return slots_info
    
    
    def _build_prompt(self, mode="", example="", dialogue_context="", ontology="", slots="", dialogue_acts="", belief_states="", database=""):
        prompt = ""
        if mode == "dst":
            instruction = self.instructions["instruction_with_slots"]
            template_variables = self.prompt_templates["template_with_slots"]
            template = PromptTemplate(input_variables= template_variables["input_variables"],
                                      template = template_variables["template"])
            prompt = template.format(instruction=instruction,
                                     slots=slots,
                                     example=example,
                                     dialogue_context=dialogue_context)
            
        elif mode == "dst_recorrect":
            instruction = self.instructions["instruction_with_slots_recorrect"]
            template_variables = self.prompt_templates["template_with_slots_recorrect"]
            template = PromptTemplate(input_variables= template_variables["input_variables"],
                                      template = template_variables["template"])            
            prompt = template.format(instruction=instruction,
                                    slots=slots,
                                    dialogue_context=dialogue_context,
                                    belief_states=belief_states)
            
        elif mode == "database_query":
            instruction = self.instructions["instruction_query_database"]
            template_variables = self.prompt_templates["template_query_database"]
            template = PromptTemplate(input_variables= template_variables["input_variables"],
                                      template = template_variables["template"])
            prompt = template.format(instruction=instruction,
                                     belief_states=belief_states)
            
        elif mode == "response_generation":
            example = self.config["EXAMPLES"]["response_generation"]
            
            instruction = self.instructions["instruction_response_generation"]
            template_variables = self.prompt_templates["template_response_generation"]
            template = PromptTemplate(input_variables = template_variables["input_variables"],
                                      template = template_variables["template"])
            prompt = template.format(instruction=instruction,
                                     example=example,
                                     dialogue_context=dialogue_context)
        elif mode == "e2e":
            instruction = self.instructions["instruction_e2e"]
            template_variables = self.prompt_templates["template_e2e"]
            template = PromptTemplate(input_variables = template_variables["input_variables"],
                                      template = template_variables["template"])
            prompt = template.format(instruction=instruction,
                                     database=database,
                                     dialogue_context=dialogue_context)

        else:
            raise ValueError("'mode' should be one of: [dst, dst_recorrect, database_query, response_generation, e2e]")
        
        return prompt


class MWOZ_Dataset(PromptConstructor):
    def __init__(self,
                 config,
                 data_args):
        PromptConstructor.__init__(self, config)
        self.dataset = {"id":[],
                        "dialogue_id":[],
                        "dialogue_context":[],
                        "turn":[],
                        "prompt_dst":[],
                        "prompt_dst_update":[],
                        "prompt_rg":[],
                        "prompt_e2e":[],
                        "domains":[],
                        "turn_domain":[],
                        "gold_turn_bs":[],
                        "gold_bs":[],
                        "gold_act":[],
                        "gold_response":[],
                        "gold_database_result":[],
                        }
        
        print("Loading data...")
        self.all_data, self.testfiles, self.system_acts, self.ontology = self._get_mwoz_data(data_args.mwoz_path)
        print("Loading databases...")
        self.dbs_lexicalized = self._get_dbs_lexicalized(data_args.mwoz_path, data_args.db_format_type)
        self.idx = 0
        self.dialog_history_limit_dst = data_args.dialog_history_limit_dst
        self.dialog_history_limit_rg = data_args.dialog_history_limit_rg
        self.dialog_history_limit_e2e = data_args.dialog_history_limit_e2e
        self.single_domain_only = data_args.single_domain_only
        self.with_slot_description = data_args.with_slot_description
        self.with_slot_domain_diff = data_args.with_slot_domain_diff
        self.with_all_slots = data_args.with_all_slots
        self.all_domains = ["restaurant", "taxi", "hotel", "train", "attraction"]

        print("Processing mwoz...")
        for sample in tqdm(self.all_data):
            if sample in self.testfiles:
                dialogue_log = self.all_data[sample]["log"]
                self._process_dialogue_log(sample=sample,
                                           dialogue_log=dialogue_log)

        self.dataset = pd.DataFrame(self.dataset)
        if self.single_domain_only:
            for index, row in tqdm(self.dataset.iterrows()):
                if "sng" not in row["dialogue_id"].lower():
                    self.dataset.drop(index, inplace=True)

        for index, row in self.dataset.iterrows():
            if row["turn_domain"] == "":
                self.dataset.loc[index, 'turn_domain'] = row["domains"][0]

                    
    def _get_mwoz_data(self, mwoz_path):
        data_path = os.path.join(mwoz_path, "data.json")
        testListFile_path = os.path.join(mwoz_path, "testListFile.txt")
        system_acts_path = os.path.join(mwoz_path, "system_acts.json")
        ontology_path = os.path.join(mwoz_path, "ontology.json")

        with open(data_path, "r") as f:
            all_data = json.load(f)
            
        with open(testListFile_path, "r") as f:
            testfiles = f.read()
        testfiles = testfiles.split("\n")
        
        with open(system_acts_path, "r") as f:
            system_acts = json.load(f)
            
        with open(ontology_path, "r") as f:
            ontology = json.load(f)
            
        return all_data, testfiles, system_acts, ontology
    
    def _get_dbs_lexicalized(self, mwoz_path, format_type):
        domains = ["restaurant", "hotel", "train", "attraction"]
        keep_data = {"restaurant":["address", "area", "food", "name", "pricerange", "phone", "postcode"],
                    "attraction":["name", "area", "address", "type", "postcode"],
                    "hotel":["name", "address", "area", "phone", "postcode", "pricerange", "stars"],
                    "train":["departure", "destination"]}
        dbs_lexicalized = {}
        for domain in domains:
            db_path = os.path.join(mwoz_path, f"{domain}_db.json")
            with open(db_path, "r") as f:
                db_data = json.load(f)

            db_lexicalized = []
            if format_type == "1":
                for row in db_data:
                    row_keep = []
                    for key in keep_data[domain]:
                        if key in row:
                            row_keep.append(f"{key}: {row[key]}")
                    db_lexicalized.append(", ".join(row_keep))
            
            elif format_type == "2":
                #more concise db to fit in context length limit
                db_lexicalized.append(", ".join(keep_data[domain]))
                for row in db_data:
                    row_keep = []
                    for key in keep_data[domain]:
                        if key in row:
                            row_keep.append(f"{row[key]}")
                    db_lexicalized.append(", ".join(row_keep))
                    # db_lexicalized.append(", ".join([f"{row[key]}" for key in keep[domain]]))
            dbs_lexicalized[domain] = "\n".join(set(db_lexicalized))

        return dbs_lexicalized
    
    def _process_dialogue_log(self, sample, dialogue_log):

        dialog_history_memory_dst = []
        dialog_history_memory_rg = []
        dialog_history_memory_e2e = []
        dialog_history_dst = ""
        dialog_history_rg = ""
        dialog_history_e2e = ""
        turn_domain = ""
        domains = self._get_domains_from_log(dialogue_log)
        slots = self._get_slots_from_domains(domains=domains, 
                                             ontology=self.ontology,
                                             with_slot_description=self.with_slot_description,
                                             with_slot_domain_diff=self.with_slot_domain_diff,
                                             with_all_slots=self.with_all_slots) # or all
        if self.dialog_history_limit_dst == 0:
            example = self.examples["dst_dh0"]
        else:
            example = self.examples["dst_dh-1"]

        for turn_nb, turn in enumerate(dialogue_log):

            if turn_nb % 2 == 0:
                speaker = "USER"
            else:
                speaker = "SYSTEM"
            
            utterance = f"""{speaker}: {turn["text"]}\n"""
            dialog_act = turn["dialog_act"]
            cur_system_act = self.system_acts[sample.split(".")[0]][str((turn_nb//2)+1)]
            
            dialogue_context_dst = dialog_history_dst + utterance
            prompt_dst = self._build_prompt(mode="dst",
                                            slots=slots,
                                            example=example,
                                            dialogue_context=dialogue_context_dst)
            
            lexicalized_act = self._lexicalize_act(cur_system_act)
            dialogue_context_rg = dialog_history_rg + utterance + f"ACT:{lexicalized_act}\nSYSTEM:"
            prompt_rg = self._build_prompt(mode="response_generation",
                                            dialogue_context=dialogue_context_rg)
            
            dialogue_context_e2e = dialog_history_e2e + utterance + "SYSTEM:"
    
            turn_domain = self._get_domain_from_turn(turn_domain, cur_system_act)
            if turn_domain and turn_domain != "taxi":
                database = self.dbs_lexicalized[turn_domain]
            else:
                database = ""
            prompt_e2e = self._build_prompt(mode="e2e",
                                            database=database,
                                            dialogue_context=dialogue_context_e2e).replace("\n\n\n", "\n")

            dialog_history_dst, dialog_history_memory_dst = self._update_dialogue_memory(utterance, 
                                                                                         dialogue_log, 
                                                                                         self.dialog_history_limit_dst, 
                                                                                         dialog_history_memory_dst)
            dialog_history_rg, dialog_history_memory_rg = self._update_dialogue_memory(utterance, 
                                                                                       dialogue_log, 
                                                                                       self.dialog_history_limit_rg,
                                                                                       dialog_history_memory_rg)
            dialog_history_e2e, dialog_history_memory_e2e = self._update_dialogue_memory(utterance, 
                                                                                         dialogue_log, 
                                                                                         self.dialog_history_limit_e2e, 
                                                                                         dialog_history_memory_e2e) 
                
            metadata = turn["metadata"]
            bspn = {}
            if metadata:
                for domain in domains:
                    for k, v in metadata[domain].items():
                        for slot, value in v.items():
                            if isinstance(value, str) and value not in ["", "not mentioned", "none"]:
                                bspn[domain+"-"+slot] = value
            self.idx += 1
            if turn_nb % 2 == 0:
                self.dataset["gold_turn_bs"].append(dialog_act)
                self.dataset["dialogue_context"].append(dialogue_context_dst)
                self.dataset["gold_database_result"].append(None) 
                self.dataset["turn"].append(turn_nb//2)
                self.dataset["domains"].append(domains)
                self.dataset["id"].append(self.idx//2)
                self.dataset["dialogue_id"].append(sample)
                self.dataset["prompt_dst"].append(prompt_dst)
                self.dataset["prompt_dst_update"].append(prompt_dst)
                self.dataset["prompt_rg"].append(prompt_rg)
                self.dataset["prompt_e2e"].append(prompt_e2e)
                self.dataset["turn_domain"].append(turn_domain)
            else:
                self.dataset["gold_response"].append(utterance)
                self.dataset["gold_bs"].append(bspn)
                self.dataset["gold_act"].append(dialog_act)

    def _update_dialogue_memory(self, utterance, dialogue_log, dialog_history_limit, dialog_history_memory):
        if dialog_history_limit != 0:
            if dialog_history_limit == -1:
                dialog_history_limit = len(dialogue_log)
            if len(dialog_history_memory) >= dialog_history_limit:
                dialog_history_memory.pop(0)
            dialog_history_memory.append(utterance)

        dialog_history = "".join(dialog_history_memory)
        return dialog_history, dialog_history_memory
    
    def _lexicalize_act(self, act):
        if act == "No Annotation":
            return "None"
        
        lexicalized_acts = []
        lexicalize_mapping = {"leave": "leave time",
                              "arrive":"arrival time",
                              "departure":"departure place",
                              "post":"postcode",
                              "addr":"address"}

        for act, slot_values in act.items():


            if "request" in act.lower():
                requests = []
                for (slot, value) in slot_values:
                    slot = slot.lower()
                    if slot in lexicalize_mapping:
                        slot = lexicalize_mapping[slot]
                    if slot == "none":
                        break
                    else:
                        requests.append(slot)
                if requests:
                    lexicalized_act = "Request the user about " + ", ".join(requests) + "."
                    lexicalized_acts.append(lexicalized_act)

            elif "recommend" in act.lower():
                recommends = []
                for (slot, value) in slot_values:
                    slot, value = slot.lower(), value.lower()
                    if slot in lexicalize_mapping:
                        slot = lexicalize_mapping[slot]
                    if slot == "none":
                        break
                    else:
                        recommends.append(value)
                if recommends:
                    lexicalized_act = "Recommend the user for " + ", ".join(recommends) + "."
                    lexicalized_acts.append(lexicalized_act)

            elif "inform" in act.lower():
                informs = []
                for (slot, value) in slot_values:
                    slot, value = slot.lower(), value.lower()
                    if slot in lexicalize_mapping:
                        slot = lexicalize_mapping[slot]
                    if slot == "none":
                        break
                    else:
                        informs.append(f"the {slot} is {value}")
                if informs:
                    lexicalized_act = "Inform the user that " + ", ".join(informs) + "."  
                    lexicalized_acts.append(lexicalized_act)

            else:
                pass
        if lexicalized_acts:
            return " ".join(lexicalized_acts)
        else:
            return "None"
        
    def _get_domain_from_turn(self, domain, act):
        for k in act:
            turn_domain = k.lower().split("-")[0]
            if turn_domain in self.all_domains:
                return turn_domain
        return domain
            

    def _get_domains_from_log(self, dialogue_log):
        domains = []
        for log in dialogue_log:
            for domain_act in log["dialog_act"]:
                domain = domain_act.split("-")[0].lower()
                if domain in self.all_domains and domain not in domains:
                    domains.append(domain)
        return domains
                
                
def evaluate_dst(results_df, vocal=True, save_path=None):
    global_turns = 0    
    global_jga = 0
    results_single_domain = {"taxi":{"turns":0, "correct_turns_jga":0, "correct_slots":0, "total_slots":0, "slot_f1":0},
                            "restaurant":{"turns":0, "correct_turns_jga":0, "correct_slots":0, "total_slots":0, "slot_f1":0},
                            "hotel":{"turns":0, "correct_turns_jga":0, "correct_slots":0, "total_slots":0, "slot_f1":0},
                            "train":{"turns":0, "correct_turns_jga":0, "correct_slots":0, "total_slots":0, "slot_f1":0},
                            "attraction":{"turns":0, "correct_turns_jga":0, "correct_slots":0, "total_slots":0, "slot_f1":0},
                            "all":{"global_turns":0, "global_f1":0}}
    
    for _, row in results_df.iterrows():
        unpacked_gold = unpack_belief_states(row["gold_bs"], "gold")
        unpacked_pred = unpack_belief_states(row["preds"], "pred")
        domains = row["domains"]
        if isinstance(domains, str):
            domains = ast.literal_eval(domains)

        if set(unpacked_gold)==set(unpacked_pred):
            global_jga += 1
            if len(domains) == 1:
                results_single_domain[domains[0]]["correct_turns_jga"] += 1

        gold_values = [gold.split("-")[1] for gold in unpacked_gold]
        pred_values = [pred.split("-")[1] for pred in unpacked_pred]
        F1, recall, precision = compute_prf(gold_values, pred_values)
        if len(domains) == 1:
            results_single_domain[domains[0]]["slot_f1"] += F1
            results_single_domain[domains[0]]["turns"] += 1
        results_single_domain["all"]["global_f1"] += F1
        results_single_domain["all"]["global_turns"] += 1
        global_turns += 1

    total_single_domain_jga = 0
    total_single_domain_turns = 0
    for domain in results_single_domain:
        if domain == "all":
            continue
        domain_slot_f1 = results_single_domain[domain]["slot_f1"]
        domain_jga = results_single_domain[domain]["correct_turns_jga"]
        domain_turns = results_single_domain[domain]["turns"]
        total_single_domain_jga += domain_jga
        total_single_domain_turns += domain_turns
        results_single_domain[domain]["JGA"] = domain_jga/domain_turns
        results_single_domain[domain]["SLOT-F1"] = domain_slot_f1/domain_turns

        if vocal:
            print(f"""For {domain}, JGA: {results_single_domain[domain]["JGA"]} - SLOT-F1: {results_single_domain[domain]["SLOT-F1"]}""")
    jga_single_domain_average = total_single_domain_jga/total_single_domain_turns
    jga_average = global_jga/global_turns    
    slot_f1_average = results_single_domain["all"]["global_f1"] / results_single_domain["all"]["global_turns"]
    if vocal:
        print(f"""Average JGA in single domain samples only: {jga_single_domain_average}""")
        print(f"""Average JGA overall: {jga_average}""")
        print(f"""Average Slot F1 Overall: {slot_f1_average}""")

    results = results_single_domain
    results["JGA_single_domain_average"] = jga_single_domain_average
    results["JGA_average"] = jga_average

    return results


def completion(model, prompt):            
    completion = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
    response = completion.choices[0].message.content.strip()
    return response    

In [3]:
model_args = ModelArguments()
data_args = DataArguments()
model_args.model_name_or_path_agent = "openai/gpt-3.5-turbo"
data_args.single_domain_only = False
data_args.dialog_history_limit_dst = 0
data_args.dialog_history_limit_e2e = -1
data_args.dialog_history_limit_rg = -1
data_args.with_slot_domain_diff = False
data_args.with_all_slots = False
data_args.with_slot_description = False

In [4]:
mwoz = MWOZ_Dataset(CONFIG, data_args)
dataset = mwoz.dataset

Loading data...
Loading databases...
Processing mwoz...


100%|██████████████████████████████████████████████████████████████████████████████████████| 10438/10438 [00:02<00:00, 3960.57it/s]


In [31]:
def get_subset_multi(df):
    df['domain_length'] = df['domains'].apply(lambda x: len(x))
    filtered_df = df.loc[(df['domain_length'] == 2)].head(500).append(df.loc[(df['domain_length'] == 3)].head(500))
    return filtered_df

dataset = get_subset_multi(dataset)

/tmp/ipykernel_1792/3716848604.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filtered_df = df.loc[(df['domain_length'] == 2)].head(500).append(df.loc[(df['domain_length'] == 3)].head(500))


In [50]:
vars(data_args)

{'dataset_name': None,
 'dataset_names': None,
 'root_data_path': './data',
 'mwoz_path': '/home/willy/instructod/MultiWOZ_2.1/',
 'dialog_history_limit_dst': 0,
 'dialog_history_limit_dst_recorrect': 0,
 'dialog_history_limit_rg': -1,
 'dialog_history_limit_e2e': 5,
 'single_domain_only': False,
 'with_slot_description': False,
 'with_req_inf_differentiation': False,
 'with_all_slots': True,
 'debug_mode': False,
 'start_idx': 0,
 'save_path': 'results/',
 'save_every': 5,
 'db_format_type': '1'}

## Getting database as prompt

In [17]:
import json
domain = "attraction"
format = "1"
db = json.load(open(f"/home/willy/instructod/MultiWOZ_2.1/{domain}_db.json", "r"))
db_slots = []
for sample in db:
    for k, v in sample.items():
        if k not in db_slots:
            db_slots.append(k)

In [10]:
ontology = json.load(open("/home/willy/instructod/MultiWOZ_2.1/ontology.json", "r"))
slots = list(ontology.keys())
slots = [slot for slot in slots if "attraction" in slot]

In [11]:
slots

['attraction-semi-area', 'attraction-semi-name', 'attraction-semi-type']

In [14]:
ontology["attraction-semi-area"]

['north', 'centre|west', 'east', 'south', 'centre', 'west', 'dontcare']

In [19]:
print(db_slots)

['address', 'area', 'entrance fee', 'id', 'location', 'name', 'openhours', 'phone', 'postcode', 'pricerange', 'type']


In [18]:
keep = {"restaurant":["name", "area", "food", "pricerange", "phone", "postcode"],
        "attraction":["name", "area", "type", "postcode"],
        "train":["departure", "destination", "leaveAt", "arriveBy", "day"]}
if format == "1":
    for row in db:
        row_keep = []
        for key in keep[domain]:
            if key in row:
                row_keep.append(f"{key}: {row[key]}")
        print(", ".join(row_keep))
                
elif format == "2":
    print(", ".join(keep[domain]))
    for row in db:
        row_keep = []
        for key in keep[domain]:
            if key in row:
                row_keep.append(f"{row[key]}")
        print(", ".join(row_keep))

# for row in db:
#     row_keep = []
#     for key in keep[domain]:
        

name: abbey pool and astroturf pitch, area: east, type: swimmingpool, postcode: cb58nt
name: adc theatre, area: centre, type: theatre, postcode: cb58as
name: all saints church, area: centre, type: architecture, postcode: cb58bs
name: ballare, area: centre, type: nightclub, postcode: cb23na
name: broughton house gallery, area: centre, type: museum, postcode: cb11ln
name: byard art, area: south, type: museum, postcode: cb21sj
name: cafe jello gallery, area: west, type: museum, postcode: cb30af
name: camboats, area: east, type: boat, postcode: cb58sx
name: cambridge and county folk museum, area: west, type: museum, postcode: cb30aq
name: cambridge arts theatre, area: centre, type: theatre, postcode: cb23pj
name: cambridge artworks, area: east, type: museum, postcode: cb13ef
name: cambridge book and print gallery, area: west, type: museum, postcode: cb39ey
name: cambridge contemporary art, area: centre, type: museum, postcode: cb21su
name: cambridge museum of technology, area: east, type: 

In [19]:
#Need to decide which keys per domain

In [46]:
file_path="/home/willy/instructod/MultiWOZ_2.1/train_db.json"
data = json.loads(Path(file_path).read_text())
df = pd.DataFrame(data)
df.head(2)

,arriveBy,day,departure,destination,duration,leaveAt,price,trainID
0,05:51,monday,cambridge,london kings cross,51 minutes,05:00,23.60 pounds,TR7075
1,07:51,monday,cambridge,london kings cross,51 minutes,07:00,23.60 pounds,TR2289


In [44]:
df["n"][0]

nan

## Using SQL database

In [22]:
import sqlite3
attraction_db = json.load(open("/home/willy/instructod/MultiWOZ_2.1/attraction_db.json"))

In [38]:
# Connect to the database
conn = sqlite3.connect('attraction.db')
cursor = conn.cursor()

# Create the attraction table
cursor.execute("DROP TABLE IF EXISTS attraction")
cursor.execute("""CREATE TABLE attraction
                  (id INT PRIMARY KEY,
                   name TEXT,
                   address TEXT,
                   area TEXT,
                   postcode TEXT,
                   phone TEXT,
                   type TEXT,
                   entrance_fee TEXT,
                   price_range TEXT)""")

# Insert data into the table
for attraction in attraction_db:
    id = int(attraction["id"])
    name = attraction["name"]
    address = attraction["address"]
    area = attraction["area"]
    postcode = attraction["postcode"]
    phone = attraction["phone"]
    type = attraction["type"]
    entrance_fee = attraction["entrance fee"]
    price_range = attraction["pricerange"]
    
    cursor.execute("""
        INSERT INTO attraction VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (id, name, address, area, postcode, phone, type, entrance_fee, price_range))

# Commit the changes and close the connection
conn.commit()
conn.close()

In [46]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
# from langchain.sql_database import SQLDatabase
from langchain import SQLDatabase
from langchain.llms.openai import OpenAI
# from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor

llm = OpenAI(temperature=0)
# llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

db = SQLDatabase.from_uri("sqlite:////home/willy/instructod/attraction.db",
                         include_tables=['attraction'],
                         sample_rows_in_table_info=3)

toolkit = SQLDatabaseToolkit(db=db,
                             llm=llm)

agent_sql = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    max_iterations=5,
    verbose=True
)

In [47]:
print(db.table_info)


CREATE TABLE attraction (
	id INTEGER, 
	name TEXT, 
	address TEXT, 
	area TEXT, 
	postcode TEXT, 
	phone TEXT, 
	type TEXT, 
	entrance_fee TEXT, 
	price_range TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from attraction table:
id	name	address	area	postcode	phone	type	entrance_fee	price_range
1	abbey pool and astroturf pitch	pool way, whitehill road, off newmarket road	east	cb58nt	01223902088	swimmingpool	?	?
2	adc theatre	park street	centre	cb58as	01223300085	theatre	?	?
3	all saints church	jesus lane	centre	cb58bs	01223452587	architecture	free	free
*/


## Using dataframe agent directly

In [7]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.llms.openai import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.agents import AgentType

import json
from pathlib import Path
from pprint import pprint

from src.e2e.e2e_utils import E2E_InstrucTOD


file_path="/home/willy/instructod/MultiWOZ_2.1/restaurant_db.json"
data = json.loads(Path(file_path).read_text())
df = pd.DataFrame(data)
# df = df.drop(columns=["location"])
df = df.drop(columns=["location", "type", "introduction", "signature", "id"])


llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)
# llm = OpenAI(temperature=0)

agent_df = create_pandas_dataframe_agent(llm, df, max_iterations=5, verbose=True)

/home/willy/miniconda3/envs/instructod/lib/python3.9/site-packages/langchain/llms/openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/willy/miniconda3/envs/instructod/lib/python3.9/site-packages/langchain/llms/openai.py:740: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [6]:
# Multi domain InstrucTOD
# instructod = E2E_InstrucTOD(CONFIG,
#                             model_args,
#                             data_args,
#                             dataset)
all_dbs = []
domains = ["attraction", "hotel", "restaurant", "train"]
for domain in domains:
    file_path=os.path.join(data_args.mwoz_path, f"{domain}_db.json")
    data = json.loads(Path(file_path).read_text())
    df = pd.DataFrame(data)
    if domain == "attraction":
        df = df.drop(columns=["location"])
    elif domain == "hotel":
        df = df.drop(columns=["location", "price", "n"])
        df['stars'] = df['stars'].astype(int)
        df = df[['name'] + [col for col in df.columns if col != 'name']]
    elif domain == "restaurant":
        df = df.drop(columns=["location", "type", "introduction", "signature", "id"])
        # df = df.rename(columns={'food': 'cuisine'})
        df = df[['name'] + [col for col in df.columns if col != 'name']]
    elif domain == "train":
        pass
    all_dbs.append(df)
    
agent_df = create_pandas_dataframe_agent(llm, all_dbs, max_iterations=5, verbose=True)

In [17]:
dummy_df = pd.DataFrame({"name":["The Inkshot", "Tattoo Party", "Pro Tattoo"],
                         "location":["123 avenue street", "92 street pop", "63 flash avenue"],
                         "colored ink":["yes", "no", "yes"],
                         "body parts":["head", "arm", "leg"],
                         "review":["good", "bad", "neutral"],
                         "laser removal":["yes", "yes", "yes"],})
agent_df = create_pandas_dataframe_agent(llm, dummy_df, max_iterations=5, verbose=False)

In [5]:
for idx, row in dataset.iterrows():
    if row["turn_domain"] == "attraction":
        prompt = dataset["prompt_e2e"][idx].split("\n\n")
        print(f"{prompt[0]}\n\n{prompt[-1]}")
        print("-------")

Generate the answer of the SYSTEM in the following conversation between a USER and a SYSTEM in a task-oriented dialogue setting. You can either request more details to the user that is available in the knowledge base to complete the goal, or simply answer the user's request. Do not provide multiple choice for the user to choose, just recommend one, and generate nothing other that the SYSTEM reply. Use the following knowledge base to interact with the user:

USER: Could you help me find a place for entertainment? Something on the east side of town. 
SYSTEM:
-------
Generate the answer of the SYSTEM in the following conversation between a USER and a SYSTEM in a task-oriented dialogue setting. You can either request more details to the user that is available in the knowledge base to complete the goal, or simply answer the user's request. Do not provide multiple choice for the user to choose, just recommend one, and generate nothing other that the SYSTEM reply. Use the following knowledge 

In [6]:
prompt_e2e = list(dataset[dataset["turn_domain"] == "attraction"]["prompt_e2e"])
dialogue_contexts = []
for prompt in prompt_e2e:
    prompt = prompt.split("\n\n")
    dialogue_contexts.append(f"{prompt[-1]}")
gold_responses = list(dataset[dataset["turn_domain"] == "attraction"]["gold_response"])
print(len(dialogue_contexts), len(gold_responses))
print("Dialogue context: ", dialogue_contexts[0])
print("----")
print("Gold: ", gold_responses[0])

47 47
Dialogue context:  USER: Could you help me find a place for entertainment? Something on the east side of town. 
SYSTEM:
----
Gold:  SYSTEM: Cherry Hinton Hall and Grounds is on the east side of town, would you like their phone number? 



In [8]:
dialogue_context ="""USER: Yeas, what to recommend if I want to see good architecture in the west part of town?
SYSTEM: Unfortunately there is no good architecture on the west end but I can look in other parts of town if you want
USER: What about a museum?""" #Ok

# dialogue_context = """USER: Yeah, I'm looking for an entertaining tourist attraction, can point me in the direction of some places to check out?
# SYSTEM: I have about 5 different entertainment venues if that is what you are looking for. Do you have a preference on the area its located in? 
# USER: No preference, please just pick one and give me the postcode and address.""" #good

# dialogue_context ="""USER: I am staying in Cambridge soon and would like to stay at a and b guest house.
# SYSTEM: Sure, how many days and how many people?
# USER: we are staying 6 people for 4 nights starting from Tuesday. i need the reference number
# SYSTEM: Your booking is successful! Your reference number is IIGRA0MI. Do you need anything else?
# USER: Yeas, what to recommend if I want to see good architecture in the west part of town?
# SYSTEM: Unfortunately there is no good architecture on the west end but I can look in other parts of town if you want
# USER: What about a museum? 
# SYSTEM: What part of town there are none in the west. 
# USER: There are no museums in the west at all?
# SYSTEM: sorry about that, there are actually seven in that area.
# USER: Great, can I get the postcode, entrance fee and address of one of them? """ Good

# dialogue_context = """USER: I am staying in the centre of town for the weekend, what is there to do there?
# SYSTEM: We have several things to do! Architecture, colleges, museums...What type of attraction are you most interested in?
# USER: It doesn't matter but can you recommend one and give me the entrance fee?"""

# dialogue_context = """USER: Hello, I would like to find a hotel that includes free parking. 
# SYSTEM: Most of the hotels in town offer free parking. Is there a certain area you'd like to stay in, or do you have a price range in mind?
# USER: Yes. The centre would be nice and also free wifi.
# SYSTEM: The University Arms is an expensive, 4 star hotel with free wifi. Comparatively, the Alexander Bed and Breakfast is a cheaply priced guesthouse, also 4 stars.
# USER: Please book me some rooms for The University Arms to accommodate 8 people for 3 nights starting on wednesday. Can you also provide me the reference number after you book? 
# SYSTEM: Your reference number is X5NY66ZV.
# USER: Thank you. Can you please help me find a place to go in town in the same area as the hotel? Preferably a college.
# SYSTEM: There are no colleges close to the area you are requesting, would you like to chose another destination?
# USER: I believe there are some colleges in the centre of town. Can you please check again?""" #Good

# dialogue_context = """USER: Hello, I would like to find a hotel that includes free parking. 
# SYSTEM: Most of the hotels in town offer free parking. Is there a certain area you'd like to stay in, or do you have a price range in mind?
# USER: Yes. The centre would be nice and also free wifi.
# SYSTEM: The University Arms is an expensive, 4 star hotel with free wifi. Comparatively, the Alexander Bed and Breakfast is a cheaply priced guesthouse, also 4 stars.
# USER: Please book me some rooms for The University Arms to accommodate 8 people for 3 nights starting on wednesday. Can you also provide me the reference number after you book? 
# SYSTEM: Your reference number is X5NY66ZV.
# USER: Thank you. Can you please help me find a place to go in town in the same area as the hotel? Preferably a college.
# SYSTEM: There are no colleges close to the area you are requesting, would you like to chose another destination?
# USER: I believe there are some colleges in the centre of town. Can you please check again? 
# SYSTEM: I am sorry but I rechecked and there are no colleges in that area.  Is there anything else that I can do for you?
# USER: Hmmm, I was looking for the phone number and postcode of a college in the centre area. Could you help me with that?
# SYSTEM: I have 5 colleges in the centre area. What specific college are you looking for?
# USER: I'm looking for cambridge."""


#PROMPTS
prompt_bs_template = """Suppose you have access to a database with all the information provided in INFORMATION, what do you need to query to the databse in order to reply to the user in the following conversation?
INFORMATION: address, area, name, phone, postcode, pricerange, entrance fee, food, internet, parking, stars, arriveby, day, departure, destination, leaveat, price.

You can follow these examples:
USER: I need fruits.
SYSTEM: Do you have any preferences?
USER: Yes, apples if possible. How expensive and how many are there?

Need: Information about pricerange and and count for apple

USER: Cool, that asnwers my question.
SYSTEM: I am happy to help. Anything else needed?
USER: I'm done, thanks!

Need: Information about closing the conversation

USER: I'm looking for a popular zoo around here
SYSTEM: There are multiple zoos I can recommend. Any preference on the location?
USER: I want it to be in the west part of town

Need: Information about zoo in the west

{dialogue_context} 

Need:"""

prompt_rg_template = """In a task oriented dialogue setting, generate a natural and helpful SYSTEM response to the USER query in the conversation provided in CONTEXT. You should follow the information provided in ACT to generate this answer, and do not mention that you are referring to a dataframe:

You can follow this example:
CONTEXT: 
USER: I need a place to fish
SYSTEM: Any preference in the type of fish?
USER: Preferably salmons, but sardines are also fine

ACT: blue lake, 37th avenue

SYSTEM: How about blue lake, 37th avenue in that case?


CONTEXT:
{dialogue_context}

ACT: {dialog_act}

SYSTEM: """

prompt_bs = PromptTemplate(
    input_variables=["dialogue_context"],
    template=prompt_bs_template,
)

prompt_rg = PromptTemplate(
    input_variables=["dialog_act", "dialogue_context"],
    template=prompt_rg_template,
)

In [15]:
import random
random_idx = random.randint(0, len(dialogue_contexts))
dialogue_context = dialogue_contexts[random_idx]
gold_response = gold_responses[random_idx]
print(dialogue_context)
print("-----")
print(gold_response)

NameError: name 'dialogue_contexts' is not defined

### Dataframe Agent

In [9]:
#Random Generation
random_idx = random.randint(0, len(dialogue_contexts))
# random_idx = 304 #Doesn't work
# random_idx = 457
# random_idx = 1232
dialogue_context = dialogue_contexts[random_idx]
gold_response = gold_responses[random_idx]
print(f"Index: {random_idx}")

print("CONTEXT: ", dialogue_context[:-9])
print("=========")
prompt = prompt_bs.format(dialogue_context=dialogue_context[:-9])
# print(prompt)
# print("=========")
output = completion("gpt-3.5-turbo", prompt)
print("User Needs to query database: ", output)

with get_openai_callback() as cb:
    try:
        # query_df = agent_df.run(f"Only give a few random suggestions maximum: {output}") #Use fake intermediary belief state
        query_df = agent_df.run(f"If there are many fitting this criteria, pick a few to propose: {output}") #Use fake intermediary belief state
    except ValueError as e:
        response = str(e)
        if not response.startswith("Could not parse LLM output: `"):
            raise e
        query_df = response.removeprefix("Could not parse LLM output: `").removesuffix("`")
    # query_df = agent({"input":f"What are some possible suggestions according to these constraints:\nNeed: {output}"})
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

print("Information retrieved from database: ", query_df)
if query_df == "Agent stopped due to iteration limit or time limit.":
    query_df = "There is nothing that fits the criteria. Ask for more information."
prompt = prompt_rg.format(dialogue_context=dialogue_context[:-9],
                          dialog_act=query_df)
# print(prompt)
# print("=========")
response = completion("gpt-3.5-turbo", prompt)
print("-----")
print("Gold response: ", gold_response)
print("Final response: ", response)

Index: 7
CONTEXT:  USER: I'm looking for some info on an attraction called Kettle's Yard
User Needs to query database:  Information about Kettle's Yard, such as address, phone number, and entrance fee.
Total Tokens: 2503
Prompt Tokens: 2339
Completion Tokens: 164
Total Cost (USD): $0.005006
Information retrieved from database:  There is no information about Kettle's Yard in the dataframe.
-----
Gold response:  SYSTEM: Absolutely! The entrance is free! It is located in the west. The address is castle street! 

Final response:  I'm sorry, but I couldn't find any information about Kettle's Yard in our system. Is there anything else I can help you with?


In [9]:
#Interaction
def interactive(domain="attraction"):
    dialogue_context = ""
    utterance = ""
    while utterance not in ["q", "quit"]:
        utterance = input("Type:")
        print(f"USER: {utterance}")
        if utterance == "quit" or utterance == "q":
            return dialogue_context
        dialogue_context += "USER: " + utterance
        prompt = prompt_bs.format(dialogue_context=dialogue_context)

        output = completion("gpt-3.5-turbo", prompt)
        print(f"Query database about: {output}")

        with get_openai_callback() as cb:
            try:
                # query_df = agent_df.run(f"Only give a few random suggestions maximum: {output}") #Use fake intermediary belief state
                query_df = agent_df.run(f"If there are many fitting this criteria, pick a few to propose: {output}") #Use fake intermediary belief state
                # query_df = agent_df.run(f"Give only one suggestion maximum: {output}") #Use fake intermediary belief state
            except ValueError as e:
                response = str(e)
                if not response.startswith("Could not parse LLM output: `"):
                    raise e
                query_df = response.removeprefix("Could not parse LLM output: `").removesuffix("`")
            # print(f"Total Tokens: {cb.total_tokens}")
            # print(f"Prompt Tokens: {cb.prompt_tokens}")
            # print(f"Completion Tokens: {cb.completion_tokens}")
            # print(f"Total Cost (USD): ${cb.total_cost}")

        # print("Information retrieved from database: ", query_df)
        if query_df == "Agent stopped due to iteration limit or time limit.":
            query_df = "There is nothing that fits the criteria. Ask for more information."
        print(f"Query database results: {query_df}")
        prompt = prompt_rg.format(dialogue_context=dialogue_context,
                                  dialog_act=query_df)

        response = completion("gpt-3.5-turbo", prompt)
        print(f"SYSTEM: {response}")
        print("-------------------")
        dialogue_context += f"\nSYSTEM: {response}\n"

In [ ]:
dialogue_context = interactive()

Type: Hi, any british food?


USER: Hi, any british food?
Query database about: Information about food that is British


> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the food is British. Then I can select a few rows to propose.

Action: Filter the dataframe to include rows where the food is British.
Action Input: `df[df['food'] == 'british']`
Observation: Filter the dataframe to include rows where the food is British. is not a valid tool, try another one.
Thought:I need to use the `query` method to filter the dataframe.
Action: Filter the dataframe to include rows where the food is British using the `query` method.
Action Input: `df.query("food == 'british'")`
Observation: Filter the dataframe to include rows where the food is British using the `query` method. is not a valid tool, try another one.
Thought:I can use the `loc` method to filter the dataframe.
Action: Filter the dataframe to include rows where the food is British using the `loc` method.
Action

In [15]:
df.head(2)

,address,area,food,name,phone,postcode,pricerange
0,Regent Street City Centre,centre,italian,pizza hut city centre,01223323737,cb21ab,cheap
1,Finders Corner Newmarket Road,east,international,the missing sock,01223812660,cb259aq,cheap


In [12]:
dummy_df.head(4)

,name,location,colored ink,body parts,review,laser removal
0,The Inkshot,123 avenue street,yes,head,good,yes
1,Tattoo Party,92 street pop,no,arm,bad,yes
2,Pro Tattoo,63 flash avenue,yes,leg,neutral,yes


In [52]:
dialogue_context = interactive()

Type: Hi, can I get international food anywhere?


USER: Hi, can I get international food anywhere?
Query database about: Information about restaurants that serve international food.


> Entering new AgentExecutor chain...
Thought: We need to filter the dataframe containing restaurant information based on the type of food they serve.
Action: python_repl_ast
Action Input: df3[df3['food'] == 'international']
Observation:                       name                               address    area  \
1         the missing sock         Finders Corner Newmarket Road    east   
25  the varsity restaurant   35 Saint Andrews Street City Centre  centre   
33   bloomsbury restaurant  Crowne Plaza Hotel 20 Downing Street  centre   

             food        phone postcode pricerange  
1   international  01223812660  cb259aq      cheap  
25  international  01223356060   cb23ar   moderate  
33  international  08719429180   cb23dt   moderate  
Thought:Query database results: We have 3 restaurants that serve international food.
SYSTEM: Sure, we have thre

Type: Yes please


USER: Yes please
Query database about: Information about name and address of restaurants that serve international cuisine.


> Entering new AgentExecutor chain...
Thought: We need to filter the restaurant dataframe by the type of cuisine and then select the name and address columns.
Action: python_repl_ast
Action Input:
```
df_restaurants = df3[df3['food'] == 'international']
df_restaurants[['name', 'address']]
```
Observation:                       name                               address
1         the missing sock         Finders Corner Newmarket Road
25  the varsity restaurant   35 Saint Andrews Street City Centre
33   bloomsbury restaurant  Crowne Plaza Hotel 20 Downing Street
Thought:Query database results: We have successfully filtered the restaurants dataframe and selected the name and address columns.
SYSTEM: Great! The three restaurants that serve international cuisine are: Spice World, located at 123 Main Street; La Cucina, located at 456 Elm Street; and Saffron, located at

Type: That's enough thanks, are there any museum around here?


...

USER: how about the fitzwilliam one?
Query database about: Information about Fitzwilliam Museum


> Entering new AgentExecutor chain...
Thought: We need to find the row(s) in the dataframe that contain information about Fitzwilliam Museum
Action: python_repl_ast
Action Input: df[df['name'].str.contains('fitzwilliam museum', case=False)]
Observation: NameError: name 'df' is not defined
Thought:We need to define the dataframes before we can use them
Action: python_repl_ast
Action Input: import pandas as pd
Observation: 
Thought:Now we can define the dataframes
Action: python_repl_ast
Action Input: df1 = pd.read_csv('data/df1.csv')
df2 = pd.read_csv('data/df2.csv')
df3 = pd.read_csv('data/df3.csv')
df4 = pd.read_csv('data/df4.csv')
Observation: FileNotFoundError: [Errno 2] No such file or directory: 'data/df1.csv'
Thought:We need to make sure that the csv files are in the correct directory
Action: python_repl_ast
Action Input: import os
os.getcwd()
Observation: /home/willy/instructod
Thou

Type: I mean the Fitzwilliam museum that you just recommended me


USER: I mean the Fitzwilliam museum that you just recommended me
Query database about: Information about the Fitzwilliam Museum


> Entering new AgentExecutor chain...
Thought: We need to find the row(s) in the dataframe that contain information about the Fitzwilliam Museum.
Action: python_repl_ast
Action Input: df1[df1['name'].str.contains('fitzwilliam', case=False)]
Observation:                address    area entrance fee  id                    name  \
69  trumpington street  centre         free  70  the fitzwilliam museum   

                                                                                                                        openhours  \
69  from tuesday to saturday it opens from 10 a.m. to 5 p.m. while on sunday, monday and bank holidays it opens from 12 to 5 p.m.   

          phone postcode pricerange    type  
69  01223332900   cb21rb       free  museum  
Thought:We found the row that contains information about the Fitzwilliam Museum.
Final Answer: The Fitzwil

Type: q


USER: q


In [54]:
print(dialogue_context)

USER: Hi, can I get international food anywhere?
SYSTEM: Sure, we have three restaurants that serve international cuisine. Would you like me to provide you with their names and locations?
USER: Yes please
SYSTEM: Great! The three restaurants that serve international cuisine are: Spice World, located at 123 Main Street; La Cucina, located at 456 Elm Street; and Saffron, located at 789 Oak Street. Would you like me to provide you with more information about any of these restaurants?
USER: That's enough thanks, are there any museum around here?
SYSTEM: Yes, there are several museums in the area. Based on your preferences, we can recommend the Cambridge Museum of Technology, Fitzwilliam Museum, Museum of Archaeology and Anthropology, Museum of Classical Archaeology, and Scott Polar Museum. Would you like more information about any of these museums?
USER: how about the fitzwilliam one?
SYSTEM: I'm sorry, but I don't have any information on the Fitzwilliam Museum at the moment. Would you lik

### SQL Agent

In [49]:
#Random Generation
# random_idx = random.randint(0, len(dialogue_contexts))
# random_idx = 304 #Doesn't work
# random_idx = 1099
dialogue_context = dialogue_contexts[random_idx]
gold_response = gold_responses[random_idx]
print(f"Index: {random_idx}")

print("CONTEXT: ", dialogue_context[:-9])
print("=========")
prompt = prompt_bs.format(dialogue_context=dialogue_context[:-9])
# print(prompt)
# print("=========")
output = completion(model_args, prompt)
print("User Needs to query database: ", output)

with get_openai_callback() as cb:
    # try:
    #     query_df = agent_sql.run(f"Only give one or two maximum: {output}") #Use fake intermediary belief state
    # except ValueError as e:
    #     response = str(e)
    #     if not response.startswith("Could not parse LLM output: `"):
    #         raise e
    #     query_df = response.removeprefix("Could not parse LLM output: `").removesuffix("`")
    query_df = agent_sql({"input":f"What are some possible suggestions according to these constraints:\nNeed: {output}"})
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

print("Information retrieved from database: ", query_df)
if query_df == "Agent stopped due to iteration limit or time limit.":
    query_df = "There is nothing that fits the criteria. Ask for more information."
prompt = prompt_rg.format(dialogue_context=dialogue_context,
                          dialog_act=query_df)
# print(prompt)
# print("=========")
response = completion(model_args, prompt)
print("-----")
print("Gold response: ", gold_response)
print("Final response: ", response)

Index: 304
CONTEXT:  SYSTEM: The only listing we have is for Bedouin in the city centre. Would you like to try a different type of cuisine?
USER: Yeah, how about a gastropub then?
SYSTEM: Yes I have several in the city center. What price range are you ideally looking for?
USER: Moderate. Please give me a phone number, address, and postcode for the restaurant you find. Thanks! 
SYSTEM: I have the cow pizza kitchen and bar located at corn exchange street cb23qf, phone number 01223308871.
USER: Cow Pizza Kitchen? That is great. lol. Can you help me find a certain attraction now? I believe it's called Gallery at Twelve
User Needs to query database:  Information on the Gallery at Twelve attraction.


> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: 
Observation: attraction
Thought: I should query the schema of the attraction table to see what information I can get.
Action: schema_sql_db
Action Input: attraction
Observation: 
CREATE TABLE attraction (
	id INTEGE

In [40]:
dataset.head(2)

,id,dialogue_id,dialogue_context,turn,prompt_dst,prompt_dst_update,prompt_rg,prompt_e2e,domains,turn_domain,gold_turn_bs,gold_bs,gold_act,gold_response,gold_database_result
0,0,SNG0073.json,USER: I would like a taxi from Saint John's college to Pizza Hut Fen Ditton.\n,0,"Generate the belief state of the very last dialogue turn in the following conversation between a USER and a SYSTEM in a task-oriented dialogue setting. The results should be in json format following this format: {'slot1':'value1', 'slot2':'value2', etc...}. Use the slot from SLOTS to generate the belief state:\n\nSLOTS:\nleaveat, pricerange, stars, phone, postcode, id, reference, parking, address, car, price, people, name, arriveby, type, area, time, departure, food, day, internet, stay, des...","Generate the belief state of the very last dialogue turn in the following conversation between a USER and a SYSTEM in a task-oriented dialogue setting. The results should be in json format following this format: {'slot1':'value1', 'slot2':'value2', etc...}. Use the slot from SLOTS to generate the belief state:\n\nSLOTS:\nleaveat, pricerange, stars, phone, postcode, id, reference, parking, address, car, price, people, name, arriveby, type, area, time, departure, food, day, internet, stay, des...","In a task oriented dialogue setting, generate a SYSTEM response to the USER query in the conversation provided in CONTEXT. You should follow the information provided in ACT to generate this answer. Do not answer with anything other than what is provided in the dialogue act:\n\nYou can follow this example:\nUSER: How much does the banana cost?\nACT: Inform the user that the price is 10$, the promotion is 80%, and the choice is 5, and request the amount that user wants.\nSYSTEM: There are 5 to...","Generate the answer of the SYSTEM in the following conversation between a USER and a SYSTEM in a task-oriented dialogue setting. You can either request more details to the user that is available in the knowledge base to complete the goal, or simply answer the user's request. Do not provide multiple choice for the user to choose, just recommend one, and generate nothing other that the SYSTEM reply. Use the following knowledge base to interact with the user:\n\nUSER: I would like a taxi from S...",[taxi],taxi,"{'Taxi-Inform': [['Dest', 'pizza hut fen ditton'], ['Depart', 'saint john 's college']]}",[taxi] dest pizza hut fenditton depart saint johns college,"{'Taxi-Request': [['Leave', '?'], ['Arrive', '?']]}",SYSTEM: What time do you want to leave and what time do you want to arrive by?\n,None
1,1,SNG0073.json,USER: I want to leave after 17:15.\n,1,"Generate the belief state of the very last dialogue turn in the following conversation between a USER and a SYSTEM in a task-oriented dialogue setting. The results should be in json format following this format: {'slot1':'value1', 'slot2':'value2', etc...}. Use the slot from SLOTS to generate the belief state:\n\nSLOTS:\nleaveat, pricerange, stars, phone, postcode, id, reference, parking, address, car, price, people, name, arriveby, type, area, time, departure, food, day, internet, stay, des...","Generate the belief state of the very last dialogue turn in the following conversation between a USER and a SYSTEM in a task-oriented dialogue setting. The results should be in json format following this format: {'slot1':'value1', 'slot2':'value2', etc...}. Use the slot from SLOTS to generate the belief state:\n\nSLOTS:\nleaveat, pricerange, stars, phone, postcode, id, reference, parking, address, car, price, people, name, arriveby, type, area, time, departure, food, day, internet, stay, des...","In a task oriented dialogue setting, generate a SYSTEM response to the USER query in the conversation provided in CONTEXT. You should follow the information provided in ACT to generate this answer. Do not answer with anything other than what is provided in the dialogue act:\n\nYou can follow this example:\nUSER: How much does the banana cost?\nACT: Inform 

## Notes:

Current e2e setting:

Encode knowledge base in a vector base, then:
1) Extract how to interact with the knowledge base (mix of belief state - system action): e.g. "Entertainment venue information including postcode and address for one venue."
2) Query the knowledge base with extracted information: e.g. "Entertainment venue information including postcode and address for adc theatre: address - park street, postcode - cb58as."
3) Use the extracted knowledge to generate a response: e.g. "Sure, how about the ADC Theatre? It's located at Park Street and the postcode is CB5 8AS."

In [1]:
import pandas as pd
df_results = pd.read_csv("src/e2e/results/gpt-3.5-turbo_e2e_agents_full_output.csv")

In [9]:
count = 0
for idx, row in df_results.iterrows():
    if "none" in row["preds_e2e_dialog_acts"]:
        count += 1

In [10]:
count

660

In [12]:
for idx, row in df_results.iterrows():
    dialogue_context = row["prompts_e2e_query_db"].split("\n\n")[-2]
    # print("dialogue context:", dialogue_context)
    print("Act: ", row["preds_e2e_dialog_acts"] + "\n")
    print("gold: ", row["gold_response"])
    print("pred: ",row["preds"])
    print("-----")

Act:  none

gold:  SYSTEM: What time do you want to leave and what time do you want to arrive by?

pred:  Sure, I can help you with that. Would you like me to confirm the booking for you?
-----
Act:  none

gold:  SYSTEM: 
Booking completed! your taxi will be blue honda Contact number is 07218068540

pred:  Sure, I can arrange a taxi for you to leave after 17:15. What time would you like to arrive at Pizza Hut Fen Ditton?
-----
Act:  none

gold:  SYSTEM: You are welcome.  Is there anything else I can help you with today?

pred:  You're welcome! Have a great day!
-----
Act:  none

gold:  SYSTEM: you too! thank you

pred:  Thank you for using our service. Have a great day!
-----
Act:  none.

gold:  SYSTEM: Yes there is a Portuguese restaurant in Cambridge  with two different locations, would you like the addresses?

pred:  I'm sorry, I couldn't find any Portuguese restaurants in Cambridge. Would you like me to search for other types of restaurants in the area?
-----
Act:  There are no Por

In [146]:
restaurant_db = json.load(open("MultiWOZ_2.1/hotel_db.json", "r"))

In [147]:
df = pd.DataFrame(restaurant_db)

In [49]:
df.head(2)

,arriveBy,day,departure,destination,duration,leaveAt,price,trainID
0,05:51,monday,cambridge,london kings cross,51 minutes,05:00,23.60 pounds,TR7075
1,07:51,monday,cambridge,london kings cross,51 minutes,07:00,23.60 pounds,TR2289


In [130]:
df = df[['name'] + [col for col in df.columns if col != 'name']]
df.head(2)

,name,address,area,cuisine,id,introduction,location,phone,postcode,pricerange,type,signature
0,pizza hut city centre,Regent Street City Centre,centre,italian,19210,Pizza hut is a large chain with restaurants nationwide offering convenience pizzas pasta and salads to eat in or take away,"[52.20103, 0.126023]",01223323737,cb21ab,cheap,restaurant,NaN
1,the missing sock,Finders Corner Newmarket Road,east,international,30650,,"[52.21768, 0.224907]",01223812660,cb259aq,cheap,restaurant,african babooti


In [131]:
df.loc[(df['type'] == 'restaurant') & (df['cuisine'] == 'british') & (df['area'] == 'west')][['name', 'address']]

,name,address
53,saint johns chop house,21 - 24 Northampton Street
81,graffiti,Hotel Felix Whitehouse Lane Huntingdon Road
93,travellers rest,Huntingdon Road City Centre


In [133]:
file_path="MultiWOZ_2.1/restaurant_db.json"
data = json.loads(Path(file_path).read_text())
df = pd.DataFrame(data)

In [135]:
df = df.drop(columns=["location", "introduction", "signature", "id"])
df = df.rename(columns={'food': 'cuisine'})
df = df[['name'] + [col for col in df.columns if col != 'name']]

In [137]:
df.query("type == 'restaurant' and cuisine == 'british' and area == 'west'").head(2)

,name,address,area,cuisine,phone,postcode,pricerange,type
53,saint johns chop house,21 - 24 Northampton Street,west,british,01223353110,cb30ad,moderate,restaurant
81,graffiti,Hotel Felix Whitehouse Lane Huntingdon Road,west,british,01223277977,cb30lx,expensive,restaurant


In [85]:
df_e2e_agent = pd.read_csv("/home/willy/instructod/src/e2e/results/gpt-3.5-turbo_e2e_agents_full_output.csv")

In [96]:
df_e2e_agent["preds_e2e_dialog_acts"].count

<bound method Series.count of 0        none
1        none
2        none
3        none
4       none.
        ...  
1054     none
1055     none
1056     none
1057     none
1058     none
Name: preds_e2e_dialog_acts, Length: 1059, dtype: object>

In [92]:
df_e2e_agent["preds_e2e_dialog_acts"][0] == "none"

True

## Comparison

In [54]:
df_rg = pd.read_csv("/home/willy/instructod/src/RG/results/gpt-4_rg_full_output.csv")
df_e2e = pd.read_csv("/home/willy/instructod/src/e2e/results/gpt-3.5-turbo_e2e_full_output.csv")
df_e2e = pd.merge(dataset, df_e2e[["id", "preds"]], on=["id"], how="right")
df_e2e_agent = pd.read_csv("/home/willy/instructod/src/e2e/results/gpt-3.5+4-turbo_e2e_agents_full_output.csv")
df_e2e_agent = pd.merge(dataset, df_e2e_agent[["id", "preds", "preds_e2e_dialog_acts"]], on=["id"], how="right")
pptod_e2e = json.load(open("/home/willy/instructod/pptod/E2E_TOD/inference_result/base/full_training/inference_result_e2e_evaluation_inform_89.2_success_79.4_bleu_18.62_combine_score_102.92.json", "r"))
pptod_e2e = pd.DataFrame([dict(item, **{"id":idx}) for idx, item in enumerate(pptod_e2e)])
pptod_e2e = pptod_e2e.rename(columns={"resp_gen":"preds"})
# df_e2e_agent_gpt3 = pd.read_csv("/home/willy/instructod/src/e2e/results/gpt-3.5-turbo_e2e_agents_full_output.csv")
# df_e2e_agent_gpt3 = pd.merge(dataset, df_e2e_agent[["id", "preds", "preds_e2e_dialog_acts"]], on=["id"], how="right")


In [55]:
count = 0
for idx, row in df_e2e_agent.iterrows():
    sample_id = row["id"]
    row_e2e = df_e2e.loc[df_e2e["id"] == sample_id]
    row_rg = df_rg.loc[df_rg["id"] == sample_id]
    row_pptod_e2e = pptod_e2e.loc[pptod_e2e["id"] == sample_id]
    if "none" in str(row["preds_e2e_dialog_acts"]):
        continue
    print("context        :", row["dialogue_context"])
    print("gold           :", row["gold_response"])
    print("pptod          :", row_pptod_e2e["preds"].item())
    print("rg             :", row_rg["preds"].item())
    print("e2e_single     :", row_e2e["preds"].item())
    print("e2e_multi      :", row["preds"])
    print("----------")
    count += 1
print(f"Printed {count} results")
    

context        : USER: If one of them has a moderate price range please give me that address. If not tell me about Turkish restaurants instead.

gold           : SYSTEM: I have two Turkish restuarants, both in the centre and both expensive. May I recommend anatolia?

pptod          : [value_name] is a [value_price] [value_food] restaurant in the [value_area] . it is located at [value_address] .
rg             : I recommend Anatolia, a Turkish restaurant in the city centre. There are two locations, both with an expensive price range. Would you like the addresses?
e2e_single     : The Portuguese restaurant with a moderate price range is located on Regent Street, and the name is De Luca Cucina and Bar. As for Turkish restaurants, there is Anatolia on Bridge Street which has a moderate price range.
e2e_multi      : I'm sorry, but there are no Portuguese or Turkish restaurants in our system. Would you like me to search for other types of restaurants in Cambridge?
----------
context        :